In [1]:
import os
import tqdm
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d


"""
Search for Giles & Deservire Modelling considers the field overlap
"""

class planar_waveguide:
    """
    Calculates the parameters for a planar waveguide
    input:
    n_1 = cladding refractive index
    n_2 = core refractive index
    n_3 = cladding refractive index
    eps_rel_1 = Relative Permittivity cladding (1)
    eps_rel_1 = Relative Permittivity of the core (2)
    eps_rel_3 = Relative Permittivity cladding (3) 
    wavelength = wavelength in nanometers
    W = width in microns
    steps = Number of steps for the iteration
    q = Number of modes to be searched for (starting from the fundamental mode 0)    
    """
    
    def __init__(self,*,n_1 = 1.455, n_2 = 1.600, n_3 = 1.455, eps_rel_1 = 1, eps_rel_2 = 1, eps_rel_3 = 1,  
                 wavelength = 1550, W = 1.1, steps = 1e6, q = 7 ):
        def islarger(data1, data2):
            """
            Returns the larger value
            """
            if data1>data2:
                return data1
            else:
                return data2
            
        self.q = np.int64(q)
        self.n_1 = np.float64(n_1)                                 #Refractive index cladding
        self.n_2 = np.float64(n_2)                                 #Refractive index core
        self.n_3 = np.float64(n_3)                                 #Refractive index cladding
        self.larger = islarger(self.n_1,self.n_3)                  #Stores the larger cladding refractive index
        self.wavelength = np.float64(wavelength*1e-9)              #Wavelength in nanometres (converted to metres)
        self.W = np.float64(W*1e-6)                                #Width in micrometres (converted to metres)
        self.steps = np.int64(steps)                               #Number of steps
        self.eps_0 = np.float64(8.854*1e-12)                       #Permittivity of free space in vacuum
        self.mu_0 = np.float64(4*np.pi*1e-7)                       #Permeability of free space in vacuum
        self.c = np.float64(1/np.sqrt(self.eps_0 * self.mu_0))     #Speed of light in m/s
        self.f = np.float64(self.c/self.wavelength)                #Frequency in Hz in vacuum
        self.w_ang = np.float64((2*np.pi)*(self.f))                #Angular velocity rad/s in vacuum
        self.k_0 = self.w_ang / self.c                             #Wavenumber in Vacuum
        self.eps_rel_1 = np.float64(eps_rel_1)                     #Relative Permittivity of cladding 1 (Affects TM mode)
        self.eps_rel_2 = np.float64(eps_rel_2)                     #Relative Permittivity of core (2) (Affects TM mode)        
        self.eps_rel_3 = np.float64(eps_rel_3)                     #Relative Permittivity of cladding 3 (Affects TM mode)
        
        
        
    def TEmode(self):
        """
        Finds the transversal electric mode
        """
        init_matrix = np.zeros(int(self.steps))
        init_modes = np.zeros((self.q,1))
        init_modes = init_modes + np.resize(np.arange(0, self.q, 1),(self.q,1))
        init_matrix = (init_matrix) + np.linspace(self.larger, self.n_2, self.steps, endpoint = False)
        gamma_1 = self.k_0 * np.sqrt(init_matrix**2 - self.n_1**2)
        gamma_2 = self.k_0 * np.sqrt(self.n_2**2 - init_matrix**2)
        gamma_3 = self.k_0 * np.sqrt(init_matrix**2 - self.n_3**2)
        result_matrix = np.absolute(gamma_2 * self.W - np.arctan(gamma_1/gamma_2) - np.arctan(gamma_3/gamma_2) 
                                    - init_modes*np.pi)
        results = OrderedDict()
        i = 0
        for elements in result_matrix:
            if np.argmin(elements)!= 0:
                results[i] = init_matrix[np.argmin(elements)]
            elif np.argmin(elements) == 0:
                break
            i += 1
        
        x = np.linspace(0,self.W,self.steps)
        try:
            gamma_1sol = self.k_0 * np.sqrt(results[0]**2 - self.n_1**2)
            gamma_2sol = self.k_0 * np.sqrt(self.n_2**2 - results[0]**2)
            alpha = -np.arctan(gamma_1sol/gamma_2sol) + init_modes * np.pi
            fig = plt.figure()
            plt.subplot(311)
            plt.plot(x, np.cos(gamma_2sol*x + alpha[0]), color = 'b')
            plt.grid(True)
            plt.ticklabel_format(style = 'sci', axis = 'x', scilimits = (0,0))
            plt.subplot(312)
            plt.grid(True)
            plt.ylabel('Intensity of electric field')
            plt.ticklabel_format(style = 'sci', axis = 'x', scilimits = (0,0))
            gamma_1sol = self.k_0 * np.sqrt(results[1]**2 - self.n_1**2)
            gamma_2sol = self.k_0 * np.sqrt(self.n_2**2 - results[1]**2)
            alpha = -np.arctan(gamma_1sol/gamma_2sol) + init_modes * np.pi
            plt.plot(x, np.cos(gamma_2sol*x + alpha[1]), color = 'g')
            plt.subplot(313)
            gamma_1sol = self.k_0 * np.sqrt(results[2]**2 - self.n_1**2)
            gamma_2sol = self.k_0 * np.sqrt(self.n_2**2 - results[2]**2)
            alpha = -np.arctan(gamma_1sol/gamma_2sol) + init_modes * np.pi
            plt.plot(x, np.cos(gamma_2sol*x + alpha[2]), color = 'm')
            plt.grid(True)
            plt.ticklabel_format(style = 'sci', axis = 'x', scilimits = (0,0))
        except:
            pass
        finally:
            plt.xlabel('Width in micrometres')
            plt.show()
    
        return results
        
    def TMmode(self):
        """
        Finds the transversal magnetic mode
        """
        init_matrix = np.zeros(int(self.steps))
        init_modes = np.zeros((self.q,1))
        init_modes = init_modes + np.resize(np.arange(0, self.q, 1),(self.q,1))
        init_matrix = (init_matrix) + np.linspace(self.larger, self.n_2, self.steps, endpoint = False)
        gamma_1 = self.k_0 * np.sqrt(init_matrix**2 - self.n_1**2)
        gamma_2 = self.k_0 * np.sqrt(self.n_2**2 - init_matrix**2)
        gamma_3 = self.k_0 * np.sqrt(init_matrix**2 - self.n_3**2)
        a = self.eps_rel_2 / self.eps_rel_1
        b = self.eps_rel_2 / self.eps_rel_3
        result_matrix = np.absolute(gamma_2 * self.W - np.arctan(a*gamma_1/gamma_2) - np.arctan(b*gamma_3/gamma_2) 
                                    - init_modes*np.pi)
        results = OrderedDict()
        i = 0
        for elements in result_matrix:
            if np.argmin(elements)!= 0:
                results[i] = init_matrix[np.argmin(elements)]
            elif np.argmin(elements) == 0:
                break
            i += 1
        return results   
    
################################################################################################################
      
class rectangular_waveguide:
    """
    Note: n = np.sqrt(eps_rel * mu_rel), if mu_rel is 1 then eps_rel
    a = Width
    b = Depth
    """
    
    def __init__(self,*,n_1 = 1.6, 
                 n_2 = 1.455, 
                 eps_rel_1 = 1, 
                 eps_rel_2 = 1, 
                 wavelength = 980, 
                 a = 0.8, 
                 b = 0.8, 
                 steps = 1e6, 
                 p = 2, 
                 q = 2,
                 use_dispersion_formula = False,
                 enable_plot = True,
                 plot_1_mode = False,
                 contour_2d = False,
                 show_surface_plot = True,
                 surface_plot = False,
                 surface_contour = False,
                 disable_progress_bar = False,
                 transparency = 1
                ):
        self.wavelength = np.float64(wavelength*1e-9)              #Wavelength in nanometres (converted to metres)
        self.use_dispersion_formula = use_dispersion_formula       #Enable the use of wavelength dependent cladding refractive index
        self.n_2 = n_2                                             #Refractive index of the cladding (user set)
        self.n_1 = np.float64(n_1)                                 #Refractive index of the core
        self.eps_rel_1 = np.float64(eps_rel_1)                     #Relative Permittivity of core
        self.eps_rel_2 = np.float64(eps_rel_2)                     #Relative Permittivity of cladding
        self.a = np.float64(a*1e-6)                                #Half of the width in micrometres (converted to metres)
        self.b = np.float64(b*1e-6)                                #Half the height in micrometres (converted to metres)
        self.steps = np.int64(steps)                               #Number of steps
        self.p = np.int64(p)                                       #Number of modes in the x direction, starting from 1
        self.q = np.int64(q)                                       #Number of modes in the y direction, starting from 1
        self.eps_0 = np.float64(8.854*1e-12)                       #Permittivity of free space in vacuum
        self.mu_0 = np.float64(4*np.pi*1e-7)                       #Permeability of free space in vacuum
        self.c = np.float64(1/np.sqrt(self.eps_0 * self.mu_0))     #Speed of light in m/s
        self.f = np.float64(self.c/self.wavelength)                #Frequency in Hz in vacuum
        self.w_ang = np.float64((2*np.pi)*(self.f))                #Angular velocity rad/s in vacuum
        self.k_0 = self.w_ang / self.c                             #Wavenumber in Vacuum
        self.enable_plot = enable_plot                             #Saves plots in hard drive, default = True
        self.plot_1_mode = plot_1_mode                             #Only plots the fundamental mode, default = False
        self.contour_2d = contour_2d                               #Enables contours inside waveguide in 2d plots, default =False
        self.show_surface_plot = show_surface_plot                 #Displays surface plots, default = False
        self.surface_plot = surface_plot                           #Saves surface plots in hard drive, default = True
        self.surface_contour = surface_contour                     #Enables or disables surface contours, default = False
        self.transparency = transparency                           #Changes transparency 0= transparent, default = 1
        self.disable_progress_bar = disable_progress_bar           #Disables the progress bar, default = False
            
    
    def EXmode(self):
        """
        Calculates the electric field polarized in the x direction
        """
        mu_0 = self.mu_0
        eps_0 = self.eps_0
        w_ang = self.w_ang
        k_0 = self.k_0
        n_1 = self.n_1
        
        #Dispersion formula for fused silica cladding
        if self.use_dispersion_formula == True:
            wavelen = self.wavelength * 1e6                        #Wavelength in micrometres for dispersion formula
            aa = np.float64((0.6961663 * wavelen **2) / (wavelen**2 - 0.0684043**2))
            bb = np.float64((0.4079426 * wavelen**2) / (wavelen**2 - 0.1162414**2))
            cc = np.float64((0.8974794 * wavelen**2) / (wavelen**2 - 9.896161**2))
            n_2_calc = np.sqrt(aa + bb + cc + 1)                        #Refractive index of the cladding, wavelength dependent
            n_2 = n_2_calc
        else:    
            n_2 = self.n_2
            
        a = self.a
        b = self.b
        steps = self.steps
        limit = np.sqrt(k_0**2 * (n_1**2 - n_2**2))
        pi = np.pi
        p = np.resize(np.arange(1,self.p+1),(self.p,1))
        q = np.resize(np.arange(1,self.q+1),(self.q,1))
        k_x = np.resize(np.linspace(1e-6,limit,steps, endpoint = False),(1,steps))
        gamma_x = np.sqrt(k_0**2 * (n_1**2 - n_2**2) - k_x**2)
        k_y = np.resize(np.linspace(1e-6,limit,steps, endpoint = False),(1,steps))
        gamma_y = np.sqrt(k_0**2 * (n_1**2 - n_2**2) - k_y**2)
        y_sol = np.absolute(k_y * b - np.arctan(gamma_y/k_y) - 0.5 * pi * (q-1))
        i = 1
        y_sols = OrderedDict()
        for elements in tqdm.tqdm(y_sol, desc = 'Finding y_sols', ncols = 110, disable = self.disable_progress_bar):
            y_sols[i] = {'k_y':k_y[0][np.argmin(elements)], 'gamma_y':gamma_y[0][np.argmin(elements)]}
            i += 1
        
        x_sols = OrderedDict()
        #Note epsilon_relative = n**2 if mu_relative = 1 (non magnetic materials)
        for i in tqdm.tqdm(range(1,self.p+1), desc = 'Finding x_sols', ncols = 110, disable = self.disable_progress_bar ):
            g = (k_0**2 * n_1**2 - y_sols[i]['k_y']**2) * gamma_x
            h = (k_0**2 * n_2**2 - y_sols[i]['k_y']**2) * k_x
            x_sol = np.absolute(k_x * a - np.arctan(g/h) - 0.5 * pi * (i-1))
            x_sols[i] = {'k_x':k_x[0][np.argmin(x_sol)], 'gamma_x':gamma_x[0][np.argmin(x_sol)]}
        
        results = OrderedDict()
        constants = OrderedDict()
        for p, value_x in tqdm.tqdm(x_sols.items(), desc = 'Combining solutions', ncols = 110, disable = self.disable_progress_bar):
            for q, value_y in y_sols.items():
                k_x = value_x['k_x']
                gamma_x = value_x['gamma_x']
                k_y = value_y['k_y']
                gamma_y = value_y['gamma_y']
                alpha_1 = (p-1) * np.pi * 0.5
                alpha_2 = (q-1) * np.pi * 0.5
                beta = np.sqrt(k_0**2 * n_1**2 - k_x**2 - k_y**2)
                n_eff = beta / k_0
                if n_2 < n_eff <= n_1:
                    results[p,q] = {
                        'k_x':k_x, 'k_y': k_y, 
                        'gamma_x': gamma_x, 'gamma_y': gamma_y, 
                        'alpha_1':alpha_1, 'alpha_2':alpha_2,
                        'beta':beta, 'n_eff':n_eff, 'k_0':k_0,
                        'n_1':n_1, 'n_2':n_2, 'a':a, 'b':b,
                        'w_ang':w_ang, 'mu_0':mu_0, 'eps_0':eps_0
                        }
                    c_1 = 1   #Initial value of constant Corresponding to intensity
                    c_2 = c_1 #* k_y * np.sin(k_y * b + alpha_2) / gamma_y
                    c_3 = c_1 #* k_x * np.sin(k_x * a + alpha_1) / gamma_x
                    c_4 = c_1 #* k_y * np.sin(k_y * b - alpha_2) / gamma_y
                    c_5 = c_1 #* k_x * np.sin(k_x * a - alpha_1) / gamma_x                    
                    constants[p,q] = {
                        'c_1':c_1,
                        'c_2':c_2,
                        'c_3':c_3,
                        'c_4':c_4,
                        'c_5':c_5
                        }
        if self.plot_1_mode == True:
            resultkeys = ((1,1),)
        else:
            resultkeys = results.keys()
        for p,q in tqdm.tqdm(resultkeys, desc = 'Creating plots', ncols = 110, disable = self.disable_progress_bar):
            if self.enable_plot == False:
                break
            modep = p
            modeq = q
            #Plots
            #Region 1
            x = np.linspace(-a, a, 100)
            y = np.linspace(-b, b, 100)
            x, y = np.meshgrid(x,y)
            E_x = constants[modep,modeq]['c_1'] * np.cos(results[modep,modeq]['k_x'] * x -\
            results[modep,modeq]['alpha_1']) * np.cos(results[modep,modeq]['k_y'] * y -\
            results[modep,modeq]['alpha_2'])
            #Region 2
            x2 = np.linspace(-a, a, 100)
            y2 = np.linspace(b, 2 * b, 200)
            x2, y2 = np.meshgrid(x2,y2)
            E_x2 = constants[modep,modeq]['c_2'] * np.cos(results[modep,modeq]['k_x'] * x2 -\
            results[modep,modeq]['alpha_1']) * np.exp(-results[modep,modeq]['gamma_y'] * (y2 - b)) *\
            np.cos(results[modep,modeq]['k_y'] * b - results[modep,modeq]['alpha_2'])
            #Region 3
            x3 = np.linspace(a, 2 * a, 200)
            y3 = np.linspace(-b, b, 100)
            x3, y3 = np.meshgrid(x3,y3)
            E_x3 = constants[modep,modeq]['c_3'] * np.cos(results[modep,modeq]['k_x'] *\
            a - results[modep,modeq]['alpha_1']) * np.exp(-results[modep,modeq]['gamma_x'] * (x3 - a)) *\
            np.cos(results[modep,modeq]['k_y'] * y3 - results[modep,modeq]['alpha_2'])
            #Region 4
            x4 = np.linspace(-a, a, 100)
            y4 = np.linspace(-b, -2 * b, 200)
            x4, y4 = np.meshgrid(x4,y4)
            E_x4 = constants[modep,modeq]['c_4'] * np.cos(results[modep,modeq]['k_x'] * x4 -\
            results[modep,modeq]['alpha_1']) * np.exp(results[modep,modeq]['gamma_y'] * (y4 + b)) *\
            np.cos(results[modep,modeq]['k_y'] * b + results[modep,modeq]['alpha_2'])
            #Region 5
            x5 = np.linspace(-a,-2 * a, 200)
            y5 = np.linspace(-b, b, 100)
            x5, y5 = np.meshgrid(x5,y5)
            E_x5 = constants[modep,modeq]['c_5'] * np.cos(results[modep,modeq]['k_x'] *\
            a + results[modep,modeq]['alpha_1']) * np.exp(results[modep,modeq]['gamma_x'] * (x5 + a)) *\
            np.cos(results[modep,modeq]['k_y'] * y5 - results[modep,modeq]['alpha_2'])
            
            #Color bar plot
            plt.hold(True)
            z = E_x
            z2 = E_x2
            z3 = E_x3
            z4 = E_x4
            z5 = E_x5
            fig, ax = plt.subplots()
            im = ax.contourf(x,y,z, vmin = -1, vmax = 1, levels = np.linspace(-1,1,100))
            if self.contour_2d == True:
                ima = ax.contour(x, y, z, 10, colors='k')
            im2 = ax.contourf(x2, y2, z2, vmin = -1, vmax = 1, levels = np.linspace(-1,1,100))
            im3 = ax.contourf(x3, y3, z3, vmin = -1, vmax = 1, levels = np.linspace(-1,1,100))
            im4 = ax.contourf(x4, y4, z4, vmin = -1, vmax = 1, levels = np.linspace(-1,1,100))
            im5 = ax.contourf(x5, y5, z5, vmin = -1, vmax = 1, levels = np.linspace(-1,1,100))
            plt.xlabel('Width (a)')
            plt.ylabel('Depth (b)')
            ax.axhline(y=-b, xmin=0, xmax=1, linewidth=0.8, color = 'k')
            ax.axhline(y=b, xmin=0, xmax=1, linewidth=0.8, color = 'k')
            ax.axvline(x=-a, ymin=0, ymax = 1, linewidth=0.8, color='k')
            ax.axvline(x=a, ymin=0, ymax = 1, linewidth=0.8, color='k')
            fig.colorbar(im, ax=ax, label = 'Electric field intensity')
            plt.ticklabel_format(style = 'sci', axis = 'x', scilimits = (0,0))
            plt.ticklabel_format(style = 'sci', axis = 'y', scilimits = (0,0))
            #Creation of plots directory
            try:
                os.makedirs(os.getcwd()+"/plots/E_x")
            except:
                pass
            plt.savefig(os.getcwd()+"/plots/E_x/EX_flat_{}{}.png".format(modep,modeq))
            plt.close("all")
            #Surface plot
            if self.surface_plot == True:
                transparency = self.transparency #from 0 (invisible) to 1 (opaque)
                fig = plt.figure()
                ax = fig.gca(projection = '3d')
                #ax = fig.add_subplot(111, projection='3d')
                ax.set_xlabel('Width (a)')
                ax.set_ylabel('Depth (b)')
                #ax.set_zlabel('Electric field intensity')
                plt.ticklabel_format(style = 'sci', axis = 'x', scilimits = (0,0))
                plt.ticklabel_format(style = 'sci', axis = 'y', scilimits = (0,0))
                ax.plot_surface(x, y, z, cmap = cm.jet, alpha = transparency, 
                                vmin = -1, vmax = 1, rstride=5, cstride=5, linewidth = 0.5)

                if self.surface_contours == True:
                    x_offset = 2 *a 
                    y_offset = -2 * b
                    z_offset = 1
                    plt.colorbar
                    ax.contour(x, y, z, 10, zdir='z', offset = z_offset, cmap=cm.jet, vmin = -1, vmax = 1)
                    ax.contour(x, y, z, 10, zdir='x', offset = x_offset, cmap=cm.bone)
                    ax.contour(x, y, z, 10, zdir='y', offset = y_offset, cmap=cm.bone) 
                ax.plot_surface(x2, y2, z2, cmap = cm.jet, alpha = transparency, 
                                vmin = -1, vmax = 1, rstride=5, cstride=5, linewidth = 0.1)#, cmap = cm.jet, rstride=10, cstride=10, linewidth = 0.1)
                ax.plot_surface(x3, y3, z3, cmap = cm.jet, alpha = transparency, 
                                vmin = -1, vmax = 1, rstride=5, cstride=5, linewidth = 0.1)
                ax.plot_surface(x4, y4, z4, cmap = cm.jet, alpha = transparency, 
                                vmin = -1, vmax = 1, rstride=5, cstride=5, linewidth = 0.1)
                ax.plot_surface(x5, y5, z5, cmap = cm.jet, alpha = transparency, 
                                vmin = -1, vmax = 1, rstride=5, cstride=5, linewidth = 0.1)
                fig.colorbar(im, ax=ax, label = 'Electric field intensity')
                ax.set_zticklabels([]) #Removes values in the z axis
                plt.savefig(os.getcwd()+"/plots/E_x/EX_surface_{}{}.png".format(modep,modeq))
                if self.show_surface_plot == True:
                    plt.show()
                plt.close("all")

        return results, constants

    def EYmode(self):
        """
        Calculates the electric field polarized in the y direction
        """
        mu_0 = self.mu_0
        eps_0 = self.eps_0
        w_ang = self.w_ang
        k_0 = self.k_0
        n_1 = self.n_1
        
        #Dispersion formula for fused silica cladding
        if self.use_dispersion_formula == True:
            wavelen = self.wavelength * 1e6                        #Wavelength in micrometres for dispersion formula
            aa = np.float64((0.6961663 * wavelen **2) / (wavelen**2 - 0.0684043**2))
            bb = np.float64((0.4079426 * wavelen**2) / (wavelen**2 - 0.1162414**2))
            cc = np.float64((0.8974794 * wavelen**2) / (wavelen**2 - 9.896161**2))
            n_2_calc = np.sqrt(aa + bb + cc + 1)                        #Refractive index of the cladding, wavelength dependent
            n_2 = n_2_calc
        else:    
            n_2 = self.n_2
            
        a = self.a
        b = self.b
        steps = self.steps
        limit = np.sqrt(k_0**2 * (n_1**2 - n_2**2))
        pi = np.pi
        p = np.resize(np.arange(1,self.p+1),(self.p,1))
        q = np.resize(np.arange(1,self.q+1),(self.q,1))
        k_x = np.resize(np.linspace(1e-6,limit,steps, endpoint = False),(1,steps))
        gamma_x = np.sqrt(k_0**2 * (n_1**2 - n_2**2) - k_x**2)
        k_y = np.resize(np.linspace(1e-6,limit,steps, endpoint = False),(1,steps))
        gamma_y = np.sqrt(k_0**2 * (n_1**2 - n_2**2) - k_y**2)
        y_sol = np.absolute(k_y * b - np.arctan((n_1**2 * gamma_y)/(k_y* n_2**2)) - 0.5 * pi * (q-1))
        #Note epsilon_relative = n**2 if mu_relative = 1 (non magnetic materials)
        i = 1
        y_sols = OrderedDict()
        for elements in tqdm.tqdm(y_sol, desc = 'Finding y_sols', ncols = 110, disable = self.disable_progress_bar):
            y_sols[i] = {'k_y':k_y[0][np.argmin(elements)], 'gamma_y':gamma_y[0][np.argmin(elements)]}
            i += 1
        
        x_sols = OrderedDict()
        for i in tqdm.tqdm(range(1,self.p+1), desc = 'Finding x_sols', ncols = 110, disable = self.disable_progress_bar):
            g = n_2**2 *(k_0**2 * n_1**2 - y_sols[i]['k_y']**2) * gamma_x
            h = n_1**2 *(k_0**2 * n_2**2 - y_sols[i]['k_y']**2) * k_x
            x_sol = np.absolute(k_x * a - np.arctan(g/h) - 0.5 * pi * (i-1))
            x_sols[i] = {'k_x':k_x[0][np.argmin(x_sol)], 'gamma_x':gamma_x[0][np.argmin(x_sol)]}
        
        results = OrderedDict()
        constants = OrderedDict()
        for p, value_x in tqdm.tqdm(x_sols.items(), desc = 'Combining solutions', ncols = 110, disable = self.disable_progress_bar):
            for q, value_y in y_sols.items():
                k_x = value_x['k_x']
                gamma_x = value_x['gamma_x']
                k_y = value_y['k_y']
                gamma_y = value_y['gamma_y']
                alpha_1 = (p-1) * np.pi * 0.5
                alpha_2 = (q-1) * np.pi * 0.5
                beta = np.sqrt(k_0**2 * n_1**2 - k_x**2 - k_y**2)
                n_eff = beta / k_0
                if n_2 < n_eff <= n_1:
                    results[p,q] = {
                        'k_x':k_x, 'k_y': k_y, 
                        'gamma_x': gamma_x, 'gamma_y': gamma_y, 
                        'alpha_1':alpha_1, 'alpha_2':alpha_2,
                        'beta':beta, 'n_eff':n_eff, 'k_0':k_0,
                        'n_1':n_1, 'n_2':n_2, 'a':a, 'b':b,
                        'w_ang':w_ang, 'mu_0':mu_0, 'eps_0':eps_0
                        }

                    c_1 = 1   #Initial value of constant Corresponding to intensity
                    c_2 = c_1 #* k_y * np.sin(k_y * b + alpha_2) / gamma_y
                    c_3 = c_1 #* k_x * np.sin(k_x * a + alpha_1) / gamma_x
                    c_4 = c_1 #* k_y * np.sin(k_y * b - alpha_2) / gamma_y
                    c_5 = c_1 #* k_x * np.sin(k_x * a - alpha_1) / gamma_x                    

                    constants[p,q] = {
                        'c_1':c_1,
                        'c_2':c_2,
                        'c_3':c_3,
                        'c_4':c_4,
                        'c_5':c_5
                        }
        print(constants)
        if self.plot_1_mode == True:
            resultkeys = ((1,1),)
        else:
            resultkeys = results.keys()
        for p,q in tqdm.tqdm(resultkeys, desc = 'Creating plots', ncols = 110, disable = self.disable_progress_bar):
            if self.enable_plot == False:
                break
            modep = p
            modeq = q
            #Plots
            #Region 1
            x = np.linspace(-a, a, 100)
            y = np.linspace(-b, b, 100)
            x, y = np.meshgrid(x,y)
            H_x = constants[modep,modeq]['c_1'] * np.cos(results[modep,modeq]['k_x'] * x -\
            results[modep,modeq]['alpha_1']) * np.cos(results[modep,modeq]['k_y'] * y -\
            results[modep,modeq]['alpha_2'])
            #Region 2
            x2 = np.linspace(-a, a, 100)
            y2 = np.linspace(b, 2 * b, 200)
            x2, y2 = np.meshgrid(x2,y2)
            H_x2 = constants[modep,modeq]['c_2'] * np.cos(results[modep,modeq]['k_x'] * x2 -\
            results[modep,modeq]['alpha_1']) * np.exp(-results[modep,modeq]['gamma_y'] * (y2 - b)) *\
            np.cos(results[modep,modeq]['k_y'] * b - results[modep,modeq]['alpha_2'])
            #Region 3
            x3 = np.linspace(a, 2 * a, 200)
            y3 = np.linspace(-b, b, 100)
            x3, y3 = np.meshgrid(x3,y3) #page 47
            H_x3 = constants[modep,modeq]['c_3'] * np.cos(results[modep,modeq]['k_x'] *\
            a - results[modep,modeq]['alpha_1']) * np.exp(-results[modep,modeq]['gamma_x'] * (x3 - a)) *\
            np.cos(results[modep,modeq]['k_y'] * y3 - results[modep,modeq]['alpha_2'])
            #Region 4
            x4 = np.linspace(-a, a, 100)
            y4 = np.linspace(-b, -2 * b, 200)
            x4, y4 = np.meshgrid(x4,y4)
            H_x4 = constants[modep,modeq]['c_4'] * np.cos(results[modep,modeq]['k_x'] * x4 -\
            results[modep,modeq]['alpha_1']) * np.exp(results[modep,modeq]['gamma_y'] * (y4 + b)) *\
            np.cos(results[modep,modeq]['k_y'] * b + results[modep,modeq]['alpha_2'])
            #Region 5
            x5 = np.linspace(-a,-2 * a, 200)
            y5 = np.linspace(-b, b, 100)
            x5, y5 = np.meshgrid(x5,y5)
            H_x5 = constants[modep,modeq]['c_5'] * np.cos(results[modep,modeq]['k_x'] *\
            a + results[modep,modeq]['alpha_1']) * np.exp(results[modep,modeq]['gamma_x'] * (x5 + a)) *\
            np.cos(results[modep,modeq]['k_y'] * y5 - results[modep,modeq]['alpha_2'])
            
            #Color bar plot
            plt.hold(True)
            z = H_x
            z2 = H_x2
            z3 = H_x3
            z4 = H_x4
            z5 = H_x5
            fig, ax = plt.subplots()
            im = ax.contourf(x,y,z, vmin = -1, vmax = 1, levels = np.linspace(-1,1,100))
            if self.contour_2d == True:
                ima = ax.contour(x, y, z, 10, colors='k')
            im2 = ax.contourf(x2, y2, z2, vmin = -1, vmax = 1, levels = np.linspace(-1,1,100))
            im3 = ax.contourf(x3, y3, z3, vmin = -1, vmax = 1, levels = np.linspace(-1,1,100))
            im4 = ax.contourf(x4, y4, z4, vmin = -1, vmax = 1, levels = np.linspace(-1,1,100))
            im5 = ax.contourf(x5, y5, z5, vmin = -1, vmax = 1, levels = np.linspace(-1,1,100))
            plt.xlabel('Width (a)')
            plt.ylabel('Depth (b)')
            ax.axhline(y=-b, xmin=0, xmax=1, linewidth=0.8, color = 'k')
            ax.axhline(y=b, xmin=0, xmax=1, linewidth=0.8, color = 'k')
            ax.axvline(x=-a, ymin=0, ymax = 1, linewidth=0.8, color='k')
            ax.axvline(x=a, ymin=0, ymax = 1, linewidth=0.8, color='k')
            fig.colorbar(im, ax=ax, label = 'Electric field intensity')
            plt.ticklabel_format(style = 'sci', axis = 'x', scilimits = (0,0))
            plt.ticklabel_format(style = 'sci', axis = 'y', scilimits = (0,0))
            #Creation of plots directory
            try:
                os.makedirs(os.getcwd()+"/plots/E_Y")
            except:
                pass
            plt.savefig(os.getcwd()+"/plots/E_Y/EY_flat_{}{}.png".format(modep,modeq))
            plt.close("all")
            #Surface plot
            if self.surface_plot == True:
                transparency = self.transparency #from 0 (invisible) to 1 (opaque)
                fig = plt.figure()
                ax = fig.gca(projection = '3d')
                #ax = fig.add_subplot(111, projection='3d')
                ax.set_xlabel('Width (a)')
                ax.set_ylabel('Depth (b)')
                #ax.set_zlabel('Electric field intensity')
                plt.ticklabel_format(style = 'sci', axis = 'x', scilimits = (0,0))
                plt.ticklabel_format(style = 'sci', axis = 'y', scilimits = (0,0))
                ax.plot_surface(x, y, z, cmap = cm.jet, alpha = transparency, 
                                vmin = -1, vmax = 1, rstride=5, cstride=5, linewidth = 0.5)

                if self.surface_contours == True:
                    x_offset = 2 *a 
                    y_offset = -2 * b
                    z_offset = 1
                    plt.colorbar
                    ax.contour(x, y, z, 10, zdir='z', offset = z_offset, cmap=cm.jet, vmin = -1, vmax = 1)
                    ax.contour(x, y, z, 10, zdir='x', offset = x_offset, cmap=cm.bone)
                    ax.contour(x, y, z, 10, zdir='y', offset = y_offset, cmap=cm.bone) 
                ax.plot_surface(x2, y2, z2, cmap = cm.jet, alpha = transparency, 
                                vmin = -1, vmax = 1, rstride=5, cstride=5, linewidth = 0.1)#, cmap = cm.jet, rstride=10, cstride=10, linewidth = 0.1)
                ax.plot_surface(x3, y3, z3, cmap = cm.jet, alpha = transparency, 
                                vmin = -1, vmax = 1, rstride=5, cstride=5, linewidth = 0.1)
                ax.plot_surface(x4, y4, z4, cmap = cm.jet, alpha = transparency, 
                                vmin = -1, vmax = 1, rstride=5, cstride=5, linewidth = 0.1)
                ax.plot_surface(x5, y5, z5, cmap = cm.jet, alpha = transparency, 
                                vmin = -1, vmax = 1, rstride=5, cstride=5, linewidth = 0.1)
                fig.colorbar(im, ax=ax, label = 'Electric field intensity')
                ax.set_zticklabels([]) #Removes values in the z axis
                plt.savefig(os.getcwd()+"/plots/E_Y/EY_surface_{}{}.png".format(modep,modeq))
                if self.show_surface_plot == True:
                    plt.show()
                plt.close("all")

        return results, constants
#Important note: c2, c3, c4, c5 in Kawano and Kitoh's book are the cos(constant values) such as c2 = cos(k_y*b-alpha1)   
#h = rectangular_waveguide(a = 3, b= 0.056)
h = rectangular_waveguide(n_1=1.461, n_2 = 1.455, a = 10, b = 2, p = 3, q =3, wavelength = 1550) 
h.transparency = 0.8
h.use_dispersion_formula = False
h.enable_plot = False
h.disable_progress_bar = True
h.plot_1_mode = False
h.contour_2d = True
h.surface_plot = True
h.surface_contours = True
h.show_surface_plot = False
w = h.EXmode()
#j = h.EYmode()
print("done")
import pprint
print(pprint.pformat(w[0]))  
#print(pprint.pformat(j))

#Create containment factor, try 2 by 2, 4 by 4, 5 by 5 and 6 by 6 and plot vs refractive index difference in %
#Note change of the refractive index based on the wavelength

done
OrderedDict([((1, 1),
              {'a': 9.9999999999999991e-06,
               'alpha_1': 0.0,
               'alpha_2': 0.0,
               'b': 1.9999999999999999e-06,
               'beta': 5908402.228768426,
               'eps_0': 8.8539999999999992e-12,
               'gamma_x': 519600.71259908524,
               'gamma_y': 373231.37523339986,
               'k_0': 4053667.9401158621,
               'k_x': 132337.77436583539,
               'k_y': 384963.02107990196,
               'mu_0': 1.2566370614359173e-06,
               'n_1': 1.4610000000000001,
               'n_2': 1.455,
               'n_eff': 1.457544701749683,
               'w_ang': 1215271965103581.2}),
             ((2, 1),
              {'a': 9.9999999999999991e-06,
               'alpha_1': 1.5707963267948966,
               'alpha_2': 0.0,
               'b': 1.9999999999999999e-06,
               'beta': 5904020.1549372971,
               'eps_0': 8.8539999999999992e-12,
               'gamma_x': 4671

In [9]:
#Determining delta values for single mode operation
delta_value = 0.0001  #Refractive index difference starting value
delta_step_size = 0.0001  #Refractive index difference step size
b_limit = 2.5
b_value = 2.5
b_step_size = 0.5  #Length step size
a_limit = 2.5       #Width limit
a_step_size = 0.5  #Width step size
a_value = 2.5       #Starting value of a
n_2 = 1.455        #Refractive index for 1550 wavelength, note default 
skip = 0
a_b_delta_min = []
a_b_delta_max = []

while a_value <= a_limit:
    print("Currently iterating over a = {}".format(a_value))
    while b_value <= b_limit:
        print("Currently iterating over b = {}".format(b_value))
        while True:
            n_1_val = np.sqrt((n_2**2)/(1-2*delta_value))
            h = rectangular_waveguide(a = a_value, 
                                      b = b_value, 
                                      n_1 = n_1_val,
                                      n_2 = 1.455,
                                      p = 2, 
                                      q = 2,
                                      wavelength = 980) 
            h.enable_plot = False
            h.disable_progress_bar = True
            w = h.EXmode()
            if len(w[0].keys()) == 1 and skip == 0:
                n_1_val = np.sqrt((1.455**2)/(1-2*delta_value))
                a_b_delta_min.append((a_value, b_value, delta_value, n_1_val))
                skip = 1
            if len(w[0].keys())>1:
                delta_value -= delta_step_size
                n_1_val = np.sqrt((1.455**2)/(1-2*delta_value))
                a_b_delta_max.append((a_value, b_value, delta_value,n_1_val))
                break
            delta_value += delta_step_size
        delta_value = 0.0001
        b_value += b_step_size
        skip = 0
    a_value += a_step_size
    b_value = 2.5

                


Currently iterating over a = 2.5
Currently iterating over b = 2.5


In [10]:
#Writing the previous iteration's data
file = open(os.getcwd()+"/"+"0-a_b_delta_min.txt","w")
file.write("a"+","+"b"+","+"delta"+","+"n_1"+"\n")
file2 = open(os.getcwd()+"/"+"0-a_b_delta_max.txt","w")
file2.write("a"+","+"b"+","+"delta"+","+"n_1"+"\n")

for elements in a_b_delta_min:
    file.write(str(elements[0])+","+str(elements[1])+","+str(elements[2])+","+str(elements[3])+"\n")
file.close()

for elements in a_b_delta_max:
    file2.write(str(elements[0])+","+str(elements[1])+","+str(elements[2])+","+str(elements[3])+"\n")
file2.close()  

In [18]:
#Reading from file the delta and calculating containment factor
import os
import pprint
from scipy import integrate
file = open(os.getcwd()+"/"+"0-a_b_delta_min.txt","r")
lines = file.readlines()
file.close()
data = []
for line in lines:
    l = (line.rstrip("\n").split(","))
    l.extend([0,])
    data.append(l)
data = np.float64(data[1:])  #format: a, b, delta, n_1, containment
file2 = open(os.getcwd()+"/"+"0-a_b_delta_max.txt","r")
lines2 = file2.readlines()
file2.close()
data2 = []
for line in lines2:
    l = (line.rstrip("\n").split(","))
    l.extend([0,])
    data2.append(l)
data2 = np.float64(data2[1:])  #format: a, b, delta, n_1, containment

def region1(x,y,params,p,q):
    a = params[0][p,q]['a']
    b = params[0][p,q]['b']
    w_ang = params[0][p,q]['w_ang']
    mu_0 = params[0][p,q]['mu_0']
    beta = params[0][p,q]['beta']
    k_x = params[0][p,q]['k_x']
    k_y = params[0][p,q]['k_y']
    gamma_x = params[0][p,q]['gamma_x']
    gamma_y = params[0][p,q]['gamma_y']
    alpha_1 = params[0][p,q]['alpha_1']
    alpha_2 = params[0][p,q]['alpha_2']
    c_1 = params[1][p,q]['c_1']
    constant = c_1**2 * (beta**2 + k_x**2)/(2 * w_ang * mu_0 * beta)
    E_x = constant * np.cos(k_x * x - alpha_1)**2 * np.cos(k_y * y - alpha_2)**2
    return E_x


def region2(x,y,params,p,q):
    a = params[0][p,q]['a']
    b = params[0][p,q]['b']
    w_ang = params[0][p,q]['w_ang']
    mu_0 = params[0][p,q]['mu_0']
    beta = params[0][p,q]['beta']
    k_x = params[0][p,q]['k_x']
    k_y = params[0][p,q]['k_y']
    gamma_x = params[0][p,q]['gamma_x']
    gamma_y = params[0][p,q]['gamma_y']
    alpha_1 = params[0][p,q]['alpha_1']
    alpha_2 = params[0][p,q]['alpha_2']
    c_2 = params[1][p,q]['c_2']
    constant = c_2**2 * (beta**2 + k_x**2)/(2 * w_ang * mu_0 * beta)
    E_x = constant * np.cos(k_x * x - alpha_1)**2 * np.exp(-2 * gamma_y * (y-b)) *  np.cos(k_y * b - alpha_2)**2
    return E_x

def region3(x,y,params,p,q):
    a = params[0][p,q]['a']
    b = params[0][p,q]['b']
    w_ang = params[0][p,q]['w_ang']
    mu_0 = params[0][p,q]['mu_0']
    beta = params[0][p,q]['beta']
    k_x = params[0][p,q]['k_x']
    k_y = params[0][p,q]['k_y']
    gamma_x = params[0][p,q]['gamma_x']
    gamma_y = params[0][p,q]['gamma_y']
    alpha_1 = params[0][p,q]['alpha_1']
    alpha_2 = params[0][p,q]['alpha_2']
    c_3 = params[1][p,q]['c_3']
    constant = c_3**2 * (beta**2 - gamma_x**2)/(2 * w_ang * mu_0 * beta)
    E_x = constant * np.cos(k_x * a - alpha_1)**2 * np.exp(-2 * gamma_x * (x-a)) *  np.cos(k_y * y - alpha_2)**2
    return E_x

for elements in data:
    n_1_val = np.sqrt((1.455**2)/(1-2*elements[2]))
    elements[3] = n_1_val
    h = rectangular_waveguide(a = elements[0], b = elements[1], n_1 = elements[3], wavelength = 1450) 
    h.transparency = 0.8
    h.enable_plot = False
    h.disable_progress_bar =True
    w = h.EXmode()
    try:
        a = w[0][1,1]['a']
        b = w[0][1,1]['b']
        p = 1
        q = 1
        params = w
        r1int = integrate.nquad(region1, [[0,a],[0,b]],args = (params, p, q), opts = {'limit':200})
        r2int = integrate.nquad(region2, [[0,a],[b,10*b]],args = (params, p, q), opts = {'limit':200})
        r3int = integrate.nquad(region3, [[a,10*a],[0,b]],args = (params, p, q), opts = {'limit':200})
        containment = r1int[0]/(r1int[0]+r2int[0]+r3int[0])
        elements[4] = containment
    except:
        elements[4] = 0

data2 = []
for i in range(16):
    wavelen = 1450 + i*10
    h = rectangular_waveguide(a = 2.5, b = 2.5, n_1 = 1.462, wavelength = wavelen) 
    h.enable_plot = False
    h.disable_progress_bar =True
    w = h.EXmode()
    try:
        a = w[0][1,1]['a']
        b = w[0][1,1]['b']
        p = 1
        q = 1
        params = w
        r1int = integrate.nquad(region1, [[0,a],[0,b]],args = (params, p, q), opts = {'limit':200})
        r2int = integrate.nquad(region2, [[0,a],[b,10*b]],args = (params, p, q), opts = {'limit':200})
        r3int = integrate.nquad(region3, [[a,10*a],[0,b]],args = (params, p, q), opts = {'limit':200})
        containment = r1int[0]/(r1int[0]+r2int[0]+r3int[0])
        data2.append((wavelen, containment))
    except:
        data2.append((wavelen,"Exception"))

file2 = open(os.getcwd()+"/"+"0-planar_waveguide_0.45_max.txt","w")
file2.write("wavelength"+","+"containment"+"\n")
for element in data2:
    file2.write(str(element[0])+","
               +str(element[1])+","
               +"\n")
file2.close()

In [16]:
#Storing containment factor & other values previously calculated
file = open(os.getcwd()+"/"+"0-planar_waveguide_0.45_min.txt","w")
file.write("a"+","+"b"+","+"n_1"+","+"a/b"+","+"delta"+","+"containment"+"\n")
for element in data:
    file.write(str(element[0])+","
               +str(element[1])+","
               +str(element[3])+","
               +str(element[0]/element[1])
               +","+str(element[2])
               +","+str(element[4])
               +","+"\n")
file.close()

file2 = open(os.getcwd()+"/"+"0-planar_waveguide_0.45_max.txt","w")
file2.write("a"+","+"b"+","+"n_1"+","+"a/b"+","+"delta"+","+"containment"+"\n")
for element in data2:
    file2.write(str(element[0])+","
               +str(element[1])+","
               +str(element[3])+","
               +str(element[0]/element[1])
               +","+str(element[2])
               +","+str(element[4])
               +","+"\n")
file2.close()

In [41]:
#Refractive index dispersion formula for fused silica
wavelength = 0.982  #Wavelength in micrometres
a = np.float64((0.6961663 * wavelength **2) / (wavelength**2 - 0.0684043**2))
b = np.float64((0.4079426 * wavelength**2) / (wavelength**2 - 0.1162414**2))
c = np.float64((0.8974794 * wavelength**2) / (wavelength**2 - 9.896161**2))
n = np.sqrt(a + b + c + 1)
print(n)

0.999913682748


In [6]:
#Reading from file the delta and calculating containment factor
#Propagating power, page 69 of Kawano and Kitoh
import os
import pprint
from scipy import integrate
file = open(os.getcwd()+"/"+"0-planar_waveguide_0.45_max_980.txt","r")
lines = file.readlines()
file.close()
data = []
for line in lines:
    l = (line.rstrip(",\n").split(","))
    data.append(l)
data = np.float64(data[1:])  #format: a, b, n_1, delta, containment

def region1(x,y,params,p,q):
    a = params[0][p,q]['a']
    b = params[0][p,q]['b']
    w_ang = params[0][p,q]['w_ang']
    mu_0 = params[0][p,q]['mu_0']
    beta = params[0][p,q]['beta']
    k_x = params[0][p,q]['k_x']
    k_y = params[0][p,q]['k_y']
    gamma_x = params[0][p,q]['gamma_x']
    gamma_y = params[0][p,q]['gamma_y']
    alpha_1 = params[0][p,q]['alpha_1']
    alpha_2 = params[0][p,q]['alpha_2']
    c_1 = params[1][p,q]['c_1']
    constant = c_1**2 * (beta**2 + k_x**2)/(2 * w_ang * mu_0 * beta)
    E_x = constant * np.cos(k_x * x - alpha_1)**2 * np.cos(k_y * y - alpha_2)**2
    return E_x


def region2(x,y,params,p,q):
    a = params[0][p,q]['a']
    b = params[0][p,q]['b']
    w_ang = params[0][p,q]['w_ang']
    mu_0 = params[0][p,q]['mu_0']
    beta = params[0][p,q]['beta']
    k_x = params[0][p,q]['k_x']
    k_y = params[0][p,q]['k_y']
    gamma_x = params[0][p,q]['gamma_x']
    gamma_y = params[0][p,q]['gamma_y']
    alpha_1 = params[0][p,q]['alpha_1']
    alpha_2 = params[0][p,q]['alpha_2']
    c_2 = params[1][p,q]['c_2']
    constant = c_2**2 * (beta**2 + k_x**2)/(2 * w_ang * mu_0 * beta)
    E_x = constant * np.cos(k_x * x - alpha_1)**2 * np.exp(-2 * gamma_y * (y-b)) *  np.cos(k_y * b - alpha_2)**2
    return E_x

def region3(x,y,params,p,q):
    a = params[0][p,q]['a']
    b = params[0][p,q]['b']
    w_ang = params[0][p,q]['w_ang']
    mu_0 = params[0][p,q]['mu_0']
    beta = params[0][p,q]['beta']
    k_x = params[0][p,q]['k_x']
    k_y = params[0][p,q]['k_y']
    gamma_x = params[0][p,q]['gamma_x']
    gamma_y = params[0][p,q]['gamma_y']
    alpha_1 = params[0][p,q]['alpha_1']
    alpha_2 = params[0][p,q]['alpha_2']
    c_3 = params[1][p,q]['c_3']
    constant = c_3**2 * (beta**2 - gamma_x**2)/(2 * w_ang * mu_0 * beta)
    E_x = constant * np.cos(k_x * a - alpha_1)**2 * np.exp(-2 * gamma_x * (x-a)) *  np.cos(k_y * y - alpha_2)**2
    return E_x

for elements in data:
    h = rectangular_waveguide(a = elements[0], b = elements[1], n_1 = elements[2], wavelength = 1550) 
    h.transparency = 0.8
    h.enable_plot = False
    h.disable_progress_bar =True
    w = h.EXmode()
    try:
        a = w[0][1,1]['a']
        b = w[0][1,1]['b']
        p = 1
        q = 1
        params = w
        r1int = integrate.nquad(region1, [[0,a],[0,b]],args = (params, p, q), opts = {'limit':200})
        r2int = integrate.nquad(region2, [[0,a],[b,10*b]],args = (params, p, q), opts = {'limit':200})
        r3int = integrate.nquad(region3, [[a,10*a],[0,b]],args = (params, p, q), opts = {'limit':200})
        containment = r1int[0]/(r1int[0]+r2int[0]+r3int[0])
        elements[5] = containment
    except:
        elements[5] = 0


In [7]:
#Storing containment factor & other values previously calculated
file = open(os.getcwd()+"/"+"0-planar_waveguide_0.45_opt_1550.txt","w")
file.write("a"+","+"b"+","+"n_1"+","+"a/b"+","+"delta"+","+"containment"+"\n")
for element in data:
    file.write(str(element[0])+","
               +str(element[1])+","
               +str(element[2])+","
               +str(element[3])
               +","+str(element[4])
               +","+str(element[5])
               +","+"\n")
file.close()

In [11]:
file.close()

In [11]:
os.getcwd()

'/home/eduardo/Desktop/Programming/Python/0-Folders'